## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate source  \
0  2025-10-14-03-30-36 +0000    nyt   
1  2025-10-14-03-24-14 +0000    nyt   
2  2025-10-14-03-15-27 +0000    nyt   
3  2025-10-14-03-14-28 +0000    bbc   
4  2025-10-14-03-12-30 +0000    bbc   

                                               title  \
0  First Brands’ C.E.O. Steps Down, While Its Acc...   
1  Doug Lebda, LendingTree C.E.O. and Founder, Di...   
2  Several News Outlets Reject Pentagon’s Reporti...   
3  Dutch government takes control of China-owned ...   
4  Trump says he may send Tomahawk missiles to Uk...   

                                                link  
0  https://www.nytimes.com/2025/10/13/business/fi...  
1  https://www.nytimes.com/2025/10/13/business/do...  
2  https://www.nytimes.com/2025/10/13/business/pe...  
3  https://www.bbc.com/news/articles/ckgk21nng0vo...  
4  https://www.bbc.com/news/articles/c93xpqgzkv0o...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
27      trump     53
161  hostages     21
500      gaza     20
23      china     19
300    israel     17

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate  source  \
111  2025-10-13-20-53-37 +0000    wapo   
108  2025-10-13-20-57-45 +0000     bbc   
118  2025-10-13-20-45-44 +0000  nypost   
160  2025-10-13-17-58-10 +0000  nypost   
146  2025-10-13-18-40-17 +0000  nypost   

                                                 title  \
111  Trump and world leaders meet in Egypt on futur...   
108  Israeli hostages and Palestinian detainees fre...   
118  Dems largely quiet as Trump headlines historic...   
160  Trump boasts Gaza peace process could be presi...   
146  Trump trolls Norway rep over Nobel Peace Prize...   

                                                  link  score  
111  https://www.washingtonpost.com/world/2025/10/1...    138  
108  https://www.bbc.com/news/articles/c740jx07vz0o...    128  
118  https://nypost.com/2025/10/13/us-news/top-dems...    124  
160  https://nypost.com/2025/10/13/us-news/trump-sa...    121  
146  https://nypost.com/2025/10/13/us-news/trump-tr...    120

## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate  source  \
111    138  2025-10-13-20-53-37 +0000    wapo   
264     68  2025-10-13-10-31-45 +0000  nypost   
79      54  2025-10-13-21-59-00 +0000     wsj   
229     52  2025-10-13-13-11-27 +0000  nypost   
16      51  2025-10-14-01-59-18 +0000     cbc   
142     48  2025-10-13-19-03-00 +0000     wsj   
10      45  2025-10-14-02-41-46 +0000  nypost   
132     44  2025-10-13-19-54-27 +0000     bbc   
195     37  2025-10-13-16-00-00 +0000     wsj   
251     33  2025-10-13-11-17-02 +0000     bbc   

                                                 title  \
111  Trump and world leaders meet in Egypt on futur...   
264  Israeli twin brothers Ziv, Gali Berman reunite...   
79   What has Trump done as president so far? The W...   
229  Dow soars nearly 600 points after Trump, Vance...   
16   New York Times, AP, Reuters and Newsmax among ...   
142  Joel Mokyr, Philippe Aghion and Peter Howitt w...   
10   Ex-student allegedly stabs 3 staff members, pl...   
132  'Peace in the Middle East': Trump signs ceasef...   
195  The relationship between capitalism and democr...   
251  Watch: Palestinian prisoners released in West ...   

                                                  link  
111  https://www.washingtonpost.com/world/2025/10/1...  
264  https://nypost.com/2025/10/13/world-news/israe...  
79   https://www.wsj.com/politics/policy/donald-tru...  
229  https://nypost.com/2025/10/13/business/dow-fut...  
16   https://www.cbc.ca/news/world/new-york-times-a...  
142  https://www.wsj.com/economy/nobel-economics-pr...  
10   https://nypost.com/2025/10/13/us-news/ex-stude...  
132  https://www.bbc.com/news/videos/c8r0xm84e4go?a...  
195  https://www.wsj.com/politics/united-states-ame...  
251  https://www.bbc.com/news/videos/cp3d571d5xyo?a...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
